In [23]:
import numpy as np
import math
import copy
import datetime
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib import cm

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [2]:
def add_up_a_decomposition_float(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=float)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# Implementation of the arXiv Version of GER

## Order of execution: get_col_indices, first_occurrence, column_freq, find_v_list, GERESA, f_score, float_GER.

This function requires three inputs:
1. $P$ is a $2^{n} \times 2^{n}$ numpy array.
2. $z$ is a real number greater than 1.
3. stopping_error is a positive real number.

This function outputs:
1. The length of the decomposition $K$.
2. A list of positive real numbers x_list.
3. A 2D numpy array representing a list of BN matrices A_array.
4. The Frobenius distance between the sum of the output decomposition and $P$.

In [12]:
def float_GER(P, z, stopping_error):
    row_num, col_num = P.shape
    
    R = copy.deepcopy(P)
    K = 0
    x_list = []
    BN_list = []
    
    while (np.linalg.norm(R, ord='fro') >= stopping_error):
        K += 1
        B = np.min(np.max(R, axis=0))
        v_list = find_v_list(R, B)
        x = 0
        score = -math.inf
        
        for v in v_list:
            temp_A = GERESA(R, v) # temp_A should be a list of length col_num representing a BN matrix.
            
            R_subtract_v_temp_A = copy.deepcopy(R)
            for j in range(col_num):
                R_subtract_v_temp_A[temp_A[j], j] -= v
            
            temp_score = f_score(R_subtract_v_temp_A, z)
            
            if ((temp_score > score) or ((temp_score == score) and (v > x))):
                score = temp_score
                x = v
                A = temp_A
        
        for j in range(col_num):
            R[A[j], j] -= x
        
        x_list.append(x)
        BN_list.append(A)
        
    return K, x_list, np.array(BN_list), np.linalg.norm(R, ord='fro')

This function requires two inputs:
1. a real number $v$.
2. A 2D numpy array an_array consisting of real numbers.

The function outputs the column frequency of $v$ in an_array.

In [5]:
def column_freq(v, an_array):
    row_num, col_num = an_array.shape
    col_freq = 0
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_freq += 1
    
    return col_freq

This function outputs the list of positive entries of $R$ which are not greater than $B$ (upper_bound) and attain the maximum column frequency in $R$.

In [6]:
def find_v_list(R, upper_bound):
    row_num, col_num = R.shape
    classify_entries = [set() for i in range(col_num + 1)]
    
    for i in range(row_num):
        for j in range(col_num):
            if (R[i, j] > 0 and R[i, j] <= upper_bound):
                classify_entries[column_freq(R[i, j], R)].add(R[i, j])
    
    empty_set = set()
    
    for freq in range(col_num, 0, -1):
        if classify_entries[freq] != empty_set:
            return list(classify_entries[freq])

In [7]:
def GERESA(R, v):
    row_num, col_num = R.shape
    A = [0] * col_num
    R_copy = copy.deepcopy(R)
    selected_columns = []
    col_indices_v_R = get_col_indices(v, R)
    col_indices_v_R_complement = set(range(col_num)) - col_indices_v_R
    
    for j in col_indices_v_R:
        A[j] = first_occurrence(v, R[:, j])
        R_copy[A[j], j] = 0
        selected_columns.append(j)
    
    for j in col_indices_v_R_complement:
        # Compute a dictionary with keys in Larger(v, R(:, j)) and values being associated column frequencies.
        dict_KLarger_VColFreq = dict()
        for i in range(row_num):
            if R[i, j] > v:
                dict_KLarger_VColFreq[i] = column_freq(R[i, j] - v, R_copy[:, selected_columns])
        
        # Set A[j] to the right choice from range(row_num).
        A[j] = max(dict_KLarger_VColFreq.keys(), key=dict_KLarger_VColFreq.get)
        
        R_copy[A[j], j] = R[A[j], j] - v
        selected_columns.append(j)
        
    return A

This function requires two inputs:
1. a real number $v$.
2. A 2D numpy array an_array consisting of real numbers.

The function outputs the set $\text{Col_indices}(v, \text{an_array})$.

In [3]:
def get_col_indices(v, an_array):
    row_num, col_num = an_array.shape
    col_indices = []
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_indices.append(j)
    
    return set(col_indices)

This function requires two inputs:
1. A real number $v$.
2. A 1D numpy array array_1D such that v is an element of array_1D.

The function outputs the index of the first occurrence of $v$ in array_1D.

In [4]:
def first_occurrence(v, array_1D):
    length = array_1D.size
    
    for i in range(length):
        if v == array_1D[i]:
            return i
    
    return -1

This function requires two inputs:
1. An real-valued 2D array array_2D.
2. A positive real number $z > 1$.

The function outputs a score.

In [8]:
def f_score(array_2D, z):
    row_num, col_num = array_2D.shape
    score = 0
    checked_positive_entries = set()
    
    for i in range(row_num):
        for j in range(col_num):
            if (array_2D[i, j] > 0) and (not array_2D[i, j] in checked_positive_entries):
                score += z ** column_freq(array_2D[i, j], array_2D)
                checked_positive_entries.add(array_2D[i, j])
    
    return score

# Functions and class needed for my parameter sensitivity analysis

In [10]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## A function for creating a surface plot for lengths of output decompositions

Inputs:
1. epsilon_index_list is a list of integers. For example, [0, 1, 2, 3, 4] represents $\varepsilon = 0.1^{0}, 0.1^{1}, 0.1^{2}, 0.1^{3}, 0.1^{4}$.

2. z_int_part_list and z_frac_part_list are both list of integers. For example, if z_int_part_list is [2, 3, 4] and z_frac_part_list is [3, 6], then the z values we are considering are [2.3, 2.6, 3.3, 3.6, 4.3, 4.6].

3. lengths_of_decoms is a dictionary whose keys are the tuples (epsilon_index, z_int_part, z_frac_part). For example, if epsilon_index_list is [0, 1], z_int_part_list is [2, 4] and z_frac_part_list is [3, 6], then the keys of lengths_of_decoms are the elements of the set $\{ 0, 1\} \times \{ 2, 4 \} \times \{ 3, 6 \}$.

4. lengths_surf_path is the filepath to save the figure.

In [32]:
def plot_lengths_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_surf_path):
    X_keys = []
    
    for int_part in z_int_part_list:
        for frac_part in z_frac_part_list:
            X_keys.append((int_part, frac_part))
    
    X = [item[0] + 0.1 * item[1] for item in X_keys]
    
    Y_keys = sorted(epsilon_index_list, reverse=True)
    Y = [-num for num in Y_keys]
    
    X, Y = np.meshgrid(X, Y)
    
    lengths_2D = np.zeros(X.shape)
    
    for i in range(len(Y_keys)):
        for j in range(len(X_keys)):
            lengths_2D[i, j] = lengths_of_decoms[(Y_keys[i], X_keys[j][0], X_keys[j][1])]
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    
    ax.plot_surface(X, Y, lengths_2D, cmap=cm.coolwarm,
                    linewidth=0, antialiased=False)
    
    ax.invert_yaxis()
    
    ax.set_xlabel("z", labelpad=10)
    ax.set_ylabel("log10(epsilon)", labelpad=10)
    ax.set_zlabel("lengths of decompositions", labelpad=10)
    
    fig.savefig(lengths_surf_path)
    plt.close(fig)

## A function for creating a surface plot for Frobenius norms of sum of decompositions

Inputs:
1. epsilon_index_list is a list of integers. For example, [0, 1, 2, 3, 4] represents $\varepsilon = 0.1^{0}, 0.1^{1}, 0.1^{2}, 0.1^{3}, 0.1^{4}$.

2. z_int_part_list and z_frac_part_list are both list of integers. For example, if z_int_part_list is [2, 3, 4] and z_frac_part_list is [3, 6], then the z values we are considering are [2.3, 2.6, 3.3, 3.6, 4.3, 4.6].

3. fro_dists is a dictionary whose keys are the tuples (epsilon_index, z_int_part, z_frac_part). For example, if epsilon_index_list is [0, 1], z_int_part_list is [2, 4] and z_frac_part_list is [3, 6], then the keys of fro_dists are the elements of the set $\{ 0, 1\} \times \{ 2, 4 \} \times \{ 3, 6 \}$.

4. error_surf_path is the filepath to save the figure.

In [27]:
def plot_error_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_surf_path):
    X_keys = []
    
    for int_part in z_int_part_list:
        for frac_part in z_frac_part_list:
            X_keys.append((int_part, frac_part))
    
    X = [item[0] + 0.1 * item[1] for item in X_keys]
    
    Y_keys = sorted(epsilon_index_list, reverse=True)
    Y = [-num for num in Y_keys]
    
    X, Y = np.meshgrid(X, Y)
    
    log10_fro_norms_2D = np.zeros(X.shape)
    
    for i in range(len(Y_keys)):
        for j in range(len(X_keys)):
            log10_fro_norms_2D[i, j] = math.log10(fro_dists[(Y_keys[i], X_keys[j][0], X_keys[j][1])])
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    
    ax.plot_surface(X, Y, log10_fro_norms_2D, cmap=cm.coolwarm,
                    linewidth=0, antialiased=False)
    
    ax.set_xlabel("z", labelpad=10)
    ax.set_ylabel("log10(epsilon)", labelpad=10)
    ax.set_zlabel("log10 errors of decompositions", labelpad=10)
    
    fig.savefig(error_surf_path)
    plt.close(fig)

## A function for creating a 3D multi-line plot for lengths of decompositions.

Inputs:
1. epsilon_index_list is a list of integers. For example, [0, 1, 2, 3, 4] represents $\varepsilon = 0.1^{0}, 0.1^{1}, 0.1^{2}, 0.1^{3}, 0.1^{4}$.

2. z_int_part_list and z_frac_part_list are both list of integers. For example, if z_int_part_list is [2, 3, 4] and z_frac_part_list is [3, 6], then the z values we are considering are [2.3, 2.6, 3.3, 3.6, 4.3, 4.6].

3. lengths_of_decoms is a dictionary whose keys are the tuples (epsilon_index, z_int_part, z_frac_part). For example, if epsilon_index_list is [0, 1], z_int_part_list is [2, 4] and z_frac_part_list is [3, 6], then the keys of lengths_of_decoms are the elements of the set $\{ 0, 1\} \times \{ 2, 4 \} \times \{ 3, 6 \}$.

4. lengths_line_path is the filepath to save the figure.

In [33]:
def plot_lengths_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list,
                         lengths_of_decoms, lengths_line_path):
    X_keys = []
    
    for int_part in z_int_part_list:
        for frac_part in z_frac_part_list:
            X_keys.append((int_part, frac_part))
    
    X = [item[0] + 0.1 * item[1] for item in X_keys]
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    
    for epsilon_index in epsilon_index_list:
        Y = [-epsilon_index] * len(X)
        Z = []
        
        for item in X_keys:
            Z.append(lengths_of_decoms[(epsilon_index, item[0], item[1])])
        
        ax.plot(X, Y, Z)
    
    ax.invert_yaxis()
    ax.set_xlabel("z", labelpad=10)
    ax.set_ylabel("log10(epsilon)", labelpad=10)
    ax.set_zlabel("lengths of decompositions", labelpad=10)
    
    fig.savefig(lengths_line_path)
    plt.close(fig)

## A function for creating a 3D multi-line plot for Frobenius norms of sum of decompositions.

Inputs:
1. epsilon_index_list is a list of integers. For example, [0, 1, 2, 3, 4] represents $\varepsilon = 0.1^{0}, 0.1^{1}, 0.1^{2}, 0.1^{3}, 0.1^{4}$.

2. z_int_part_list and z_frac_part_list are both list of integers. For example, if z_int_part_list is [2, 3, 4] and z_frac_part_list is [3, 6], then the z values we are considering are [2.3, 2.6, 3.3, 3.6, 4.3, 4.6].

3. fro_dists is a dictionary whose keys are the tuples (epsilon_index, z_int_part, z_frac_part). For example, if epsilon_index_list is [0, 1], z_int_part_list is [2, 4] and z_frac_part_list is [3, 6], then the keys of lengths_of_decoms are the elements of the set $\{ 0, 1\} \times \{ 2, 4 \} \times \{ 3, 6 \}$.

4. error_line_path is the filepath to save the figure.

In [28]:
def plot_error_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list,
                       fro_dists, error_line_path):
    X_keys = []
    
    for int_part in z_int_part_list:
        for frac_part in z_frac_part_list:
            X_keys.append((int_part, frac_part))
    
    X = [item[0] + 0.1 * item[1] for item in X_keys]
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    
    for epsilon_index in epsilon_index_list:
        Y = [-epsilon_index] * len(X)
        Z = []
        
        for item in X_keys:
            Z.append(math.log10(fro_dists[(epsilon_index, item[0], item[1])]))
        
        ax.plot(X, Y, Z)
    
    ax.set_xlabel("z", labelpad=10)
    ax.set_ylabel("log10(epsilon)", labelpad=10)
    ax.set_zlabel("log10 errors of decompositions", labelpad=10)
    
    fig.savefig(error_line_path)
    plt.close(fig)

# Parameter Sensitivity Analysis ($8 \times 8$ float TPMs)

Then, for each of these matrices, for $\varepsilon = 1, 0.1, \ldots, 10^{-4}$, for $z = 1.1, 1.2, \ldots, 9.9, 10$,
1. Execute GER on it.
2. Save the output decomposition.
3. Plot a surface plot (z: lengths of decompositions, x: $z$-values, y: $\varepsilon$-values (log scale for y)).
4. Plot a surface plot (z: Frobenius distance to the input TPM (log scale), x: $z$-values, y: $\varepsilon$-values (log scale for y))
5. Plot several line plots (z-axis is the lengths of decompositions) on the same 3D figure (each line uses the same $\varepsilon$).
6. Plot several line plots (z-axis is the Frobenius distance to the input TPM – log scale) on the same 3D figure (each line uses the same $\varepsilon$).

In [36]:
epsilon_index_list = list(range(5))
z_int_part_list = list(range(1, 10))
z_frac_part_list = list(range(1, 11))

for k in range(1, 10):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = dict()
    fro_dists = dict()
    filepath = './random float TPMs/psa_dim8_fTPM_' + str(k) + '.npy'
    fTPM = np.load(filepath)
    
    for epsilon_index in epsilon_index_list:
        print("Start of epsilon_index =", epsilon_index)
        for z_int_part in z_int_part_list:
            for z_frac_part in z_frac_part_list:
                decomposition = float_GER(fTPM, z_int_part + 0.1 * z_frac_part, 0.1 ** epsilon_index)
                lengths_of_decoms[(epsilon_index, z_int_part, z_frac_part)] = decomposition[0]
                fro_dists[(epsilon_index, z_int_part, z_frac_part)] = decomposition[3]

                decom_path = './output decoms (float TPMs)/dim8_matrix_' + str(k) + '/epsidx_' + \
                             str(epsilon_index) + '_z_' + str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
                with open(decom_path, "w") as out:
                    json.dump(decomposition, out, cls=NumpyEncoder)
            
                if ((z_int_part in [3, 6, 9]) and (z_frac_part == 10)):
                    print("finished z =", z_int_part + 0.1 * z_frac_part)
    
    # LSP stands for "lengths surface plot".
    lengths_surf_path = './lengths surface plots (float TPMs)/LSP_dim8_fTPM_' + str(k) + '.png'
    plot_lengths_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_surf_path)
    
    # ESP stands for "errors surface plot".
    error_surf_path = './error surface plots (float TPMs)/ESP_dim8_fTPM_' + str(k) + '.png'
    plot_error_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_surf_path)
    
    # LLP stands for "lengths line plot".
    lengths_line_path = './lengths line plots (float TPMs)/LLP_dim8_fTPM_' + str(k) + '.png'
    plot_lengths_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_line_path)
    
    # ELP stands for "errors line plot".
    error_line_path = './error line plots (float TPMs)/ELP_dim8_fTPM_' + str(k) + '.png'
    plot_error_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_line_path)
    
    print("End of k =", k)
    print()

Start of k = 1      2025-04-08 00:04:22.230295
Start of epsilon_index = 0
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 1
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 2
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 3
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 4
finished z = 4.0
finished z = 7.0
finished z = 10.0
End of k = 1

Start of k = 2      2025-04-08 00:17:52.074968
Start of epsilon_index = 0
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 1
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 2
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 3
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 4
finished z = 4.0
finished z = 7.0
finished z = 10.0
End of k = 2

Start of k = 3      2025-04-08 00:30:45.273335
Start of epsilon_index = 0
finished z = 4

KeyboardInterrupt: 

The following code is for generating all the required figures after decompositions of GER with all the tested values of the parameters (z, stopping_error) have been obtained and saved into folders.

In [35]:
epsilon_index_list = list(range(5))
z_int_part_list = list(range(1, 10))
z_frac_part_list = list(range(1, 11))

for k in range(1):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = dict()
    fro_dists = dict()
    filepath = './random float TPMs/psa_dim8_fTPM_' + str(k) + '.npy'
    fTPM = np.load(filepath)
    
    for epsilon_index in epsilon_index_list:
        print("Start of epsilon_index =", epsilon_index)
        for z_int_part in z_int_part_list:
            for z_frac_part in z_frac_part_list:
                decom_path = './output decoms (float TPMs)/dim8_matrix_' + str(k) + '/epsidx_' + \
                             str(epsilon_index) + '_z_' + str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
                
                with open(decom_path) as f:
                    decomposition = json.load(f)
                
                lengths_of_decoms[(epsilon_index, z_int_part, z_frac_part)] = decomposition[0]
                fro_dists[(epsilon_index, z_int_part, z_frac_part)] = decomposition[3]
            
                if ((z_int_part in [3, 6, 9]) and (z_frac_part == 10)):
                    print("finished z =", z_int_part + 0.1 * z_frac_part)
    
    # LSP stands for "lengths surface plot".
    lengths_surf_path = './lengths surface plots (float TPMs)/LSP_dim8_fTPM_' + str(k) + '.png'
    plot_lengths_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_surf_path)
    
    # ESP stands for "errors surface plot".
    error_surf_path = './error surface plots (float TPMs)/ESP_dim8_fTPM_' + str(k) + '.png'
    plot_error_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_surf_path)
    
    # LLP stands for "lengths line plot".
    lengths_line_path = './lengths line plots (float TPMs)/LLP_dim8_fTPM_' + str(k) + '.png'
    plot_lengths_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_line_path)
    
    # ELP stands for "errors line plot".
    error_line_path = './error line plots (float TPMs)/ELP_dim8_fTPM_' + str(k) + '.png'
    plot_error_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_line_path)
    
    print("End of k =", k)
    print()

Start of k = 0      2025-04-07 23:55:57.653218
Start of epsilon_index = 0
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 1
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 2
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 3
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 4
finished z = 4.0
finished z = 7.0
finished z = 10.0
End of k = 0



# Parameter Sensitivity Analysis ($16 \times 16$ float TPMs)

Then, for each of these matrices, for $\varepsilon = 1, 0.1, \ldots, 10^{-4}$, for $z = 1.1, 1.2, \ldots, 9.9, 10$,
1. Execute GER on it.
2. Save the output decomposition.
3. Plot a surface plot (z: lengths of decompositions, x: $z$-values, y: $\varepsilon$-values (log scale for y)).
4. Plot a surface plot (z: Frobenius distance to the input TPM (log scale), x: $z$-values, y: $\varepsilon$-values (log scale for y))
5. Plot several line plots (z-axis is the lengths of decompositions) on the same 3D figure (each line uses the same $\varepsilon$).
6. Plot several line plots (z-axis is the Frobenius distance to the input TPM – log scale) on the same 3D figure (each line uses the same $\varepsilon$).

In [37]:
epsilon_index_list = list(range(5))
z_int_part_list = list(range(1, 10))
z_frac_part_list = list(range(1, 11))

for k in range(5):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = dict()
    fro_dists = dict()
    filepath = './random float TPMs/psa_dim16_fTPM_' + str(k) + '.npy'
    fTPM = np.load(filepath)
    
    for epsilon_index in epsilon_index_list:
        print("Start of epsilon_index =", epsilon_index, '    ', datetime.datetime.now())
        for z_int_part in z_int_part_list:
            for z_frac_part in z_frac_part_list:
                decomposition = float_GER(fTPM, z_int_part + 0.1 * z_frac_part, 0.1 ** epsilon_index)
                lengths_of_decoms[(epsilon_index, z_int_part, z_frac_part)] = decomposition[0]
                fro_dists[(epsilon_index, z_int_part, z_frac_part)] = decomposition[3]

                decom_path = './output decoms (float TPMs)/dim16_matrix_' + str(k) + '/epsidx_' + \
                             str(epsilon_index) + '_z_' + str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
                with open(decom_path, "w") as out:
                    json.dump(decomposition, out, cls=NumpyEncoder)
            
                if ((z_int_part in [3, 6, 9]) and (z_frac_part == 10)):
                    print("finished z =", z_int_part + 0.1 * z_frac_part)
    
    # LSP stands for "lengths surface plot".
    lengths_surf_path = './lengths surface plots (float TPMs)/LSP_dim16_fTPM_' + str(k) + '.png'
    plot_lengths_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_surf_path)
    
    # ESP stands for "errors surface plot".
    error_surf_path = './error surface plots (float TPMs)/ESP_dim16_fTPM_' + str(k) + '.png'
    plot_error_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_surf_path)
    
    # LLP stands for "lengths line plot".
    lengths_line_path = './lengths line plots (float TPMs)/LLP_dim16_fTPM_' + str(k) + '.png'
    plot_lengths_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_line_path)
    
    # ELP stands for "errors line plot".
    error_line_path = './error line plots (float TPMs)/ELP_dim16_fTPM_' + str(k) + '.png'
    plot_error_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_line_path)
    
    print("End of k =", k)
    print()

Start of k = 0      2025-04-08 01:07:08.780918
Start of epsilon_index = 0      2025-04-08 01:07:08.805146
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 1      2025-04-08 01:24:46.636159
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 2      2025-04-08 03:17:36.620852
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 3      2025-04-08 05:54:08.262576
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 4      2025-04-08 09:23:49.829163
finished z = 4.0
finished z = 7.0
finished z = 10.0
End of k = 0

Start of k = 1      2025-04-08 14:55:04.934952
Start of epsilon_index = 0      2025-04-08 14:55:04.948696
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 1      2025-04-08 15:08:42.598485
finished z = 4.0
finished z = 7.0
finished z = 10.0
Start of epsilon_index = 2      2025-04-08 17:37:32.830005
finished z = 4.0
finished z = 7.0
finished z = 10.0
Star

KeyboardInterrupt: 

# Parameter Sensitivity Analysis ($32 \times 32$ float TPMs)

Then, for each of these matrices, for $\varepsilon = 1, 0.1, \ldots, 10^{-4}$, for $z = 1.1, 1.2, \ldots, 9.9, 10$,
1. Execute GER on it.
2. Save the output decomposition.
3. Plot a surface plot (z: lengths of decompositions, x: $z$-values, y: $\varepsilon$-values (log scale for y)).
4. Plot a surface plot (z: Frobenius distance to the input TPM (log scale), x: $z$-values, y: $\varepsilon$-values (log scale for y))
5. Plot several line plots (z-axis is the lengths of decompositions) on the same 3D figure (each line uses the same $\varepsilon$).
6. Plot several line plots (z-axis is the Frobenius distance to the input TPM – log scale) on the same 3D figure (each line uses the same $\varepsilon$).

In [ ]:
epsilon_index_list = list(range(5))
z_int_part_list = list(range(1, 10))
z_frac_part_list = list(range(1, 11))

for k in range(5):
    print('Start of k =', k, '    ', datetime.datetime.now())
    lengths_of_decoms = dict()
    fro_dists = dict()
    filepath = './random float TPMs/psa_dim32_fTPM_' + str(k) + '.npy'
    fTPM = np.load(filepath)
    
    for epsilon_index in epsilon_index_list:
        print("Start of epsilon_index =", epsilon_index, '    ', datetime.datetime.now())
        for z_int_part in z_int_part_list:
            for z_frac_part in z_frac_part_list:
                decomposition = float_GER(fTPM, z_int_part + 0.1 * z_frac_part, 0.1 ** epsilon_index)
                lengths_of_decoms[(epsilon_index, z_int_part, z_frac_part)] = decomposition[0]
                fro_dists[(epsilon_index, z_int_part, z_frac_part)] = decomposition[3]

                decom_path = './output decoms (float TPMs)/dim32_matrix_' + str(k) + '/epsidx_' + \
                             str(epsilon_index) + '_z_' + str(z_int_part) + 'dot' + str(z_frac_part) + '.json'
        
                with open(decom_path, "w") as out:
                    json.dump(decomposition, out, cls=NumpyEncoder)
            
                if ((z_int_part in [3, 6, 9]) and (z_frac_part == 10)):
                    print("finished z =", z_int_part + 0.1 * z_frac_part, '      ', datetime.datetime.now())
    
    # LSP stands for "lengths surface plot".
    lengths_surf_path = './lengths surface plots (float TPMs)/LSP_dim32_fTPM_' + str(k) + '.png'
    plot_lengths_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_surf_path)
    
    # ESP stands for "errors surface plot".
    error_surf_path = './error surface plots (float TPMs)/ESP_dim32_fTPM_' + str(k) + '.png'
    plot_error_surface(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_surf_path)
    
    # LLP stands for "lengths line plot".
    lengths_line_path = './lengths line plots (float TPMs)/LLP_dim32_fTPM_' + str(k) + '.png'
    plot_lengths_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                         lengths_of_decoms, lengths_line_path)
    
    # ELP stands for "errors line plot".
    error_line_path = './error line plots (float TPMs)/ELP_dim32_fTPM_' + str(k) + '.png'
    plot_error_3Dlines(epsilon_index_list, z_int_part_list, z_frac_part_list, 
                       fro_dists, error_line_path)
    
    print("End of k =", k)
    print()